# LAB: Árvores de decisão e Bagging


## Introdução

Neste lab vamos comparar o rendimento de um classificador de árvore de decisão simples com um classificador de Bagging. Vamos fazer isso em poucos conjuntos de dados, começando por aqueles oferecidos pelo Scikit Learn.

Como já aprendemos, data science é um processo iterativo. Muitas vezes, começamos com um modelo muito simples e depois tentamos melhorar seu rendimento ou achar modelos melhores para comparar com o inicial.

Esse é exatamente o processo que vamos seguir neste laboratório. Vamos começar com um modelo simples (árvore de decisão) e depois vamos comparar seu rendimento com um modelo ensamblagem mais complexo.

Vamos usar dois conjuntos de dados:

- para classificação, vamos usar o [conjunto de dados sobre câncer de mama](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic). Tentaremos diagnosticar um câncer de mama a partir de algumas características do núcleo celular

- para regressão, vamos usar o [conjunto de dados sobre diabetes](http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf), onde tentaremos obter uma medida quantitativa da progressão de uma doença um ano depois da linha de base de 10 variáveis basais.

## Exercício

### Requisitos

1. Usando o conjunto de dados sobre câncer de mama, criar um classificador de árvore de decisão e usar cross_val_score para avaliar seu rendimento
    - Compare o rendimento com o classificador de Bagging
    - Usar pipelines e escalado para ver se o rendimento melhora
    - Explorar o espaço de parâmetros com Grid Search
- Usando o conjunto de dados sobre diabetes, criar uma árvore de regressão e compará-la com um regressor de Bagging
    - Comparação simples usando cross_val_score
    - Explorar o espaço de parâmetros com Grid Search

**Extra:**

- Repetir a análise anterior para o conjunto de dados do IMDB

### Outros recursos

- [Validação cruzada no exemplo de diabetes](http://scikit-learn.org/stable/auto_examples/exercises/plot_cv_diabetes.html)
- [paper do conjunto de dados sobre diabetes](http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf)
- [Classificador Bagging](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)
- [Grid Search](http://scikit-learn.org/stable/modules/grid_search.html)

## 1. Conjunto de dados sobre câncer de mama
Vamos começar a comparação com o conjunto de dados sobre câncer de mama.
Ele pode ser lido diretamente no scikit-learn com a função `load_breast_cancer`.

### 1.a Comparação simples
1. Ler os dados e criar X e y
- Criar uma árvore de classificação e usar cross_val_score para avaliar seu rendimento. Fixar a validação cruzada em 5-folds
- Construir um classificador de Bagging sobre um classificador de árvore e usar cross_val_score para avaliar seu rendimento. Fixar a validação cruzada em 5-folds
- Qual resultado é melhor? O resultado é significativamente diferente? Como isso pode ser interpretado?
> Resposta: usar o desvio padrão dos resultados de saída de cross_val_score

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
data = load_breast_cancer()

In [4]:
data.keys()

['target_names', 'data', 'target', 'DESCR', 'feature_names']

In [5]:
X = pd.DataFrame(data['data'], columns=data['feature_names'])
y = pd.Series(data['target'])

In [6]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
y.value_counts()

1    357
0    212
dtype: int64

In [8]:
y.value_counts()/y.count()

1    0.627417
0    0.372583
dtype: float64

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

In [10]:
dt = DecisionTreeClassifier()

In [11]:
def do_cross_val(model):
    scores = cross_val_score(model, X, y, cv=5, n_jobs=-1)
    return scores.mean(), scores.std()

do_cross_val(dt)

(0.92093882262408611, 0.023641765185628574)

In [12]:
bdt = BaggingClassifier(DecisionTreeClassifier())

In [13]:
do_cross_val(bdt)

(0.94568680261639104, 0.024717812236490867)

### 1.b Escalado com pipelines
Como é possível ver, as características não estão normalizadas. Será que o resultado vai melhorar com normalização?

1. Crie 2 pipelines, com um passo de pré-processamento de escalado e depois uma árvore de decisão e uma árvore de bagging.
- Qual resultado é melhor? O resultado é significativamente diferente? Como isso pode ser interpretado?
Os resultados são diferentes daqueles não escalados?
> Não, os resultados são os mesmos


In [14]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline

In [15]:
pipedt = make_pipeline(RobustScaler(),
DecisionTreeClassifier())
pipebdt = make_pipeline(RobustScaler(),
BaggingClassifier(DecisionTreeClassifier()))

In [16]:
do_cross_val(pipedt)

(0.92093882262408611, 0.018429649745660866)

In [17]:
do_cross_val(pipebdt)

(0.94568680261639104, 0.024717812236490867)

### 1.c Grid Search

O Grid search é uma forma excelente de melhorar o rendimento de um classificador. Vamos explorar o espaço de parâmetros de ambos os modelos e ver se podemos melhorar seu rendimento.

1. Iniciar um GridSearchCV com uma validação cruzada de 5-fold para a árvore de decisão de classificação
- Procurar alguns valores dos parâmetros para melhorar o resultado do classificador
- Usar todo o conjunto de datos X e y para o teste
- Comparar o best\_score\_ após treinar. Ele melhorou?
> Sim
- Como se compara o resultado do Grid Search da árvore de decisão com o da árvore de Bagging?
> Reposta: eles são iguais (dentro do erro), o Grid Search melhorou o resultado da árvore simples
- Iniciar um GridSearchCV com validação cruzada de 5-fold para o classificador da árvore de Bagging
- Repetir a busca
    - Considerar que é preciso cambiar os nomes dos parâmetros para o base_estimator
    - Considerar que também é preciso mudar certos parâmetros adicionais
    - Considerar que isso pode acabar tendo um espaço de busca muito grande e vai exigir muito tempo
    - Usar o parâmetro n_jobs para acelerar a busca

- O resultado para o classificador de Bagging melhora?
> Sim
- Qual resultado é melhor? O resultado é significativamente diferente? Como isso pode ser interpretado?
> O classificador de Bagging resultante do GridSearch é o melhor

In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
params = {"max_depth": [3,10,None],
"max_features": [None, "auto"],"min_samples_leaf": [1, 5, 10],"min_samples_split": [2, 10]         }
    

gsdt = GridSearchCV(dt, params, n_jobs=-1, cv=5)

In [21]:
gsdt.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_features': [None, 'auto'], 'min_samples_split': [2, 10], 'max_depth': [3, 10, None], 'min_samples_leaf': [1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [22]:
gsdt.best_params_

{'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 2}

In [23]:
gsdt.best_score_

0.94551845342706498

In [24]:
bdt.get_params()

{'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             presort=False, random_state=None, splitter='best'),
 'base_estimator__class_weight': None,
 'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': None,
 'base_estimator__max_features': None,
 'base_estimator__max_leaf_nodes': None,
 'base_estimator__min_impurity_split': 1e-07,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 2,
 'base_estimator__min_weight_fraction_leaf': 0.0,
 'base_estimator__presort': False,
 'base_estimator__random_state': None,
 'base_estimator__splitter': 'best',
 'bootstrap': True,
 'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose':

In [28]:
params = {"base_estimator__max_depth": [3,10,None],
"base_estimator__max_features": [None, "auto"],"base_estimator__min_samples_leaf": [1, 5, 10],"base_estimator__min_samples_split": [2, 10],'bootstrap_features': [False, True],'max_features': [0.5, 1.0],'max_samples': [0.5, 1.0],'n_estimators': [5, 15, 40],         }
    

gsbdt = GridSearchCV(bdt, params, n_jobs=3, cv=5)

In [29]:
gsbdt.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
        ...n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'n_estimators': [5, 15, 40], 'max_samples': [0.5, 1.0], 'base_estimator__min_samples_split': [2, 10], 'base_estimator__max_depth': [3, 10, None], 'bootstrap_features': [False, True], 'max_features': [0.5, 1.0], 'base_estimator__min_samples_leaf': [1, 5, 10], 'base_estimator__max_features': [None, 'auto']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [30]:
gsbdt.best_params_

{'base_estimator__max_depth': 3,
 'base_estimator__max_features': None,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 2,
 'bootstrap_features': True,
 'max_features': 0.5,
 'max_samples': 1.0,
 'n_estimators': 40}

In [31]:
gsbdt.best_score_

0.96836555360281196

## 2 Diabetes e regressão

O Scikit Learn tem um conjunto de dados sobre pacientes diabéticos obtidos do seguinte estudo:

http://www4.stat.ncsu.edu/~boos/var.select/diabetes.html
http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf

Foram medidos 442 pacientes diabéticos conforme 10 variáveis basais: idade, sexo, índice de massa corporal, pressão arterial média e seis medições do soro sanguíneo.

O objetivo é uma medição quantitativa da progressão da doença um ano depois da linha de base.

Repita a comparação anterior entre um DecisionTreeRegressor e uma versão de Bagging dele.

### 2.a Comparação simples
1. Ler os dados e criar X e y
- Criar uma árvore de regressão e usar cross_val_score para avaliar seu rendimento. Fixar a validação cruzada em 5-fold. Qual medida de avaliação vai ser utilizada?
> Resposta: r2
- Criar uma árvore de regressão com Bagging e usar cross_val_score para avaliar seu rendimento. Fixar a validação cruzada em 5-fold.
- Qual resultado é melhor? O resultado é significativamente diferente? Como isso pode ser interpretado?
> A árvore de regressão com Bagging é melhor   
> Resposta: usar o desvio padrão dos resultados de saída de cross_val_score

In [32]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

data = load_diabetes()
X_full = data['data']
y_full = data['target']

X, X_test, y, y_test = train_test_split( X_full, y_full, test_size=0.1, random_state=42)


In [33]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

def do_cross_val(model):
    scores = cross_val_score(model, X, y, cv=5, n_jobs=-1, scoring='r2')
    return scores.mean(), scores.std()


In [34]:
dtr = DecisionTreeRegressor()
do_cross_val(dtr)

(-0.1654913918664859, 0.16919067118361442)

In [35]:
bdtr = BaggingRegressor(DecisionTreeRegressor())
do_cross_val(bdtr)

(0.33044521695210333, 0.10305941003131509)

### 2.b Grid Search

Repetir o Grid search conforme mostrado acima:

1. Iniciar um GridSearchCV com 5-fold de validação cruzada para a árvore de regressão
- Procurar alguns valores dos parâmetros para melhorar o resultado da regressão
- Usar todo o conjunto de dados X e y para o teste
- Comparar o best\_score\_ após treinar. Ele melhorou?
> Sim
- Como se compara o resultado do Grid Search da árvore de decisão com o da árvore de Bagging?
> Reposta: eles são iguais (dentro do erro), o Grid Search melhorou o resultado da árvore simples
- Iniciar um GridSearchCV com validação cruzada de 5-fold para a árvore de regressão de Bagging
- Repetir a busca

    - Considerar que é preciso cambiar os nomes dos parâmetros para o base_estimator
    - Considerar que também é preciso testar certos parâmetros adicionais
    - Considerar que isso pode acabar tendo um espaço de busca muito grande e vai exigir muito tempo
    - Usar o parâmetro n_jobs para acelerar a busca

- O resultado para o Bagging de regressão melhora?
> Sim
- Qual resultado é melhor? O resultado é significativamente diferente? Como isso pode ser interpretado?
> O Bagging de regressão resultante do GridSearch é o melhor

In [45]:
dtr=DecisionTreeRegressor()
dtr.get_params()

{'criterion': 'mse',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_split': 1e-07,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

In [46]:
params = {"splitter": ['best', 'random'],
"max_depth": [3,5,10,20],"max_features": [None, "auto"],"min_samples_leaf": [1, 3, 7, 10],"min_samples_split": [2, 5, 7]         }
    

gsdtr = GridSearchCV(dtr, params, n_jobs=-1, cv=5, scoring='r2')

In [47]:
gsdtr.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_features': [None, 'auto'], 'splitter': ['best', 'random'], 'min_samples_split': [2, 5, 7], 'max_depth': [3, 5, 10, 20], 'min_samples_leaf': [1, 3, 7, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [48]:
gsdtr.best_params_

{'max_depth': 3,
 'max_features': None,
 'min_samples_leaf': 3,
 'min_samples_split': 7,
 'splitter': 'random'}

In [49]:
gsdtr.best_score_

0.3309184342681491

In [50]:
params = {"base_estimator__splitter": ['best', 'random'],
"base_estimator__max_depth": [3,5,10,20],"base_estimator__max_features": [None, "auto"],"base_estimator__min_samples_leaf": [1, 3, 7, 10],"base_estimator__min_samples_split": [2, 5, 7],'bootstrap_features': [False, True],'max_features': [0.5, 1.0],'max_samples': [0.5, 1.0],'n_estimators': [5, 15, 40],         }
    

gsbdtr = GridSearchCV(bdtr, params, n_jobs=-1, cv=5, scoring='r2')

In [51]:
gsbdtr.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,...n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [5, 15, 40], 'max_samples': [0.5, 1.0], 'base_estimator__min_samples_split': [2, 5, 7], 'base_estimator__max_depth': [3, 5, 10, 20], 'bootstrap_features': [False, True], 'base_estimator__splitter': ['best', 'random'], 'max_features': [0.5, 1.0], 'base_estimator__min_samples_leaf': [1, 3, 7, 10], 'base_estimator__max_features': [None, 'auto']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [52]:
gsbdtr.best_params_

{'base_estimator__max_depth': 20,
 'base_estimator__max_features': 'auto',
 'base_estimator__min_samples_leaf': 3,
 'base_estimator__min_samples_split': 2,
 'base_estimator__splitter': 'random',
 'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 0.5,
 'n_estimators': 40}

In [53]:
gsbdtr.best_score_

0.45420261411429808

Comparemos o rendimento dos seguintes modelos com o test set:
* A melhor árvore de regressão obtida com o GridSearch
* O melhor Bagging de árvores de regressão obtido com o GridSearch
* Alguma das árvores de regressão base do Bagging de árvores


In [54]:
from sklearn.metrics import r2_score

def eval_score(legend,model):
    r2=r2_score(y_test,model.predict(X_test))
    print legend,r2

eval_score("Mejor árbol de regresión:",gsdtr.best_estimator_)
eval_score("Mejor Bagging de árboles de regresión:",gsbdtr.best_estimator_)
eval_score("Un árbol base del Bagging:",gsbdtr.best_estimator_.estimators_[0])

Mejor árbol de regresión: 0.375607824805
Mejor Bagging de árboles de regresión: 0.593373844195
Un árbol base del Bagging: 0.290068715067
